# Donations to political parties in the Netherlands

By Els Engel

My first real python project. I'm trying to get an overview of donations to political parties in the Netherlands. It's not news any more, since the dataset was released in March, but it's interesting data any way.

Data: [Rijksoverheid.nl](https://www.rijksoverheid.nl/documenten/publicaties/2017/03/08/overzicht-giften-en-schulden-politieke-partijen-tweede-kamerverkiezingen) (first download link)

As you can see, it's pdf. I opened it in Adobe Reader, saved as excel and did some very minimal cleaning in Open Office. Then saved to csv.

Two points:
- Only donations higher than € 4.500 are included in the data set
- Some political parties require their politicians to contribute (part of) their income to the party. Those donations are also included (that's why the SP has so many donations)



### Import the data

In [25]:
import pandas as pd

All the donations (Dutch csv's are delimitered with ; because our decimal symbol is a ,)

In [26]:
giften = pd.read_csv("file:///Users/elsengel/Desktop/partijen.csv", sep=";")

For future use: all members of the government, so I can exclude them from the data set

In [27]:
kieslijst = pd.read_csv("file:///Users/elsengel/Desktop/kieslijst.csv", sep=";")

### This is what the messy data set looks like, time to clean

In [29]:
giften.head()

,Partij,full_name,adres_gever,bedrag,totaalbedrag,tijdvak
0,NaN,1. Volkspartij voor de Vrijheid en Democratie ...,NaN,NaN,NaN,NaN
1,NaN,Vereniging Volkspartij voor Vrijheid en Democr...,NaN,NaN,NaN,NaN
2,1. Volkspartij voor de Vrijheid en Democratie ...,Stichting Godefridus van Hees Fonds,Eikenlaan 22 4356 HE Oostkapelle,35000,NaN,12 March 2015
3,1. Volkspartij voor de Vrijheid en Democratie ...,Naam gever,Adres gever,Bedrag,Totaalbedrag,Datum bijdrage
4,1. Volkspartij voor de Vrijheid en Democratie ...,NaN,NaN,NaN,35000,Subtotaal 2015


Every page in de pdf repeated the column names, so I have to get those out of the rows

In [31]:
giften.full_name = giften.full_name.replace(to_replace="Naam gever")

If one person donated more than once, the name wasn't repeated on the next row, there's this neat function in pandas to fix that

In [32]:
giften.full_name = giften.full_name.fillna(method='ffill') 

In [33]:
giften.head()

,Partij,full_name,adres_gever,bedrag,totaalbedrag,tijdvak
0,NaN,1. Volkspartij voor de Vrijheid en Democratie ...,NaN,NaN,NaN,NaN
1,NaN,Vereniging Volkspartij voor Vrijheid en Democr...,NaN,NaN,NaN,NaN
2,1. Volkspartij voor de Vrijheid en Democratie ...,Stichting Godefridus van Hees Fonds,Eikenlaan 22 4356 HE Oostkapelle,35000,NaN,12 March 2015
3,1. Volkspartij voor de Vrijheid en Democratie ...,Stichting Godefridus van Hees Fonds,Adres gever,Bedrag,Totaalbedrag,Datum bijdrage
4,1. Volkspartij voor de Vrijheid en Democratie ...,Stichting Godefridus van Hees Fonds,NaN,NaN,35000,Subtotaal 2015


### Let's focus on last year's donations

In [34]:
giften2016 = giften[giften.tijdvak == "Subtotaal 2016"]

In [36]:
giften2016.head()

,Partij,full_name,adres_gever,bedrag,totaalbedrag,tijdvak
6,1. Volkspartij voor de Vrijheid en Democratie ...,Stichting Godefridus van Hees Fonds,NaN,NaN,35000,Subtotaal 2016
11,1. Volkspartij voor de Vrijheid en Democratie ...,Dhr. Benedictus,NaN,NaN,0,Subtotaal 2016
16,1. Volkspartij voor de Vrijheid en Democratie ...,Club van 100 Utrecht,NaN,NaN,0,Subtotaal 2016
21,1. Volkspartij voor de Vrijheid en Democratie ...,Stichting Doe meer met Utrecht,NaN,NaN,0,Subtotaal 2016
26,1. Volkspartij voor de Vrijheid en Democratie ...,Dhr. Broeders,NaN,NaN,5000,Subtotaal 2016


Remove people who donated nothing last year

In [37]:
giften2016 = giften2016[giften2016.totaalbedrag != "0"]

In [38]:
giften2016.head()

,Partij,full_name,adres_gever,bedrag,totaalbedrag,tijdvak
6,1. Volkspartij voor de Vrijheid en Democratie ...,Stichting Godefridus van Hees Fonds,NaN,NaN,35000,Subtotaal 2016
26,1. Volkspartij voor de Vrijheid en Democratie ...,Dhr. Broeders,NaN,NaN,5000,Subtotaal 2016
59,2. Partij van de Arbeid (PvdA),A. Eijsink,NaN,NaN,4836,Subtotaal 2016
63,2. Partij van de Arbeid (PvdA),A. Marcouch,NaN,NaN,5274,Subtotaal 2016
67,2. Partij van de Arbeid (PvdA),A.A. de Vries,NaN,NaN,5600,Subtotaal 2016


### What party received the most donations last year?

Of course it's SP, because all their members are required to donate their entire income to the party.

In [41]:
giften2016.Partij.value_counts().reset_index()

,index,Partij
0,4. Socialistische Partij (SP),597
1,8. GROENLINKS (GL),56
2,2. Partij van de Arbeid (PvdA),47
3,6. Democraten 66 (D66),11
4,10. Partij voor de Dieren (PvdD),7
5,7. ChristenUnie (CU),3
6,3. Partij voor de Vrijheid (PVV),3
7,1. Volkspartij voor de Vrijheid en Democratie ...,2
8,9. Staatkundig Gereformeerde Partij (SGP),1
9,11. 50PLUS,1


### Next step: filter out members of the government

### Then: see what party received the most money

### And then: who gave the biggest donations to which party?